In [46]:
import datasets
import pandas as pd

eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]
eval_ds = eval_ds.rename_columns(
    {"Question": "question", "Final answer": "true_answer", "Level": "task"}
)


def preprocess_file_paths(row):
    if len(row["file_name"]) > 0:
        row["file_name"] = "data/gaia/validation/" + row["file_name"]

    return row


eval_ds = eval_ds.map(preprocess_file_paths)

eval_df = pd.DataFrame(eval_ds)

Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


In [47]:
eval_df["task"].value_counts()

task
2    86
1    53
3    26
Name: count, dtype: int64

In [48]:
eval_df["extension"] = eval_df["file_name"].apply(lambda x: x.split(".")[-1])

In [49]:
eval_df["extension"].value_counts()

extension
          127
xlsx       13
png         8
mp3         3
pdf         3
zip         2
jpg         2
pdb         1
jsonld      1
docx        1
txt         1
pptx        1
csv         1
py          1
Name: count, dtype: int64

### Prepare submission

In [1]:
import datasets
import pandas as pd

pd.set_option("max_colwidth", None)

SET = "test"

eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")[SET]
eval_ds = eval_ds.rename_columns(
    {"Question": "question", "Final answer": "true_answer", "Level": "task"}
)
eval_df = pd.DataFrame(eval_ds)

Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


In [2]:
results = pd.read_json(
    f"output_gaia/{SET}/react_code_gpt4o_23-june_planning2_newprompt5_test.jsonl",
    lines=True,
)

In [3]:
from scripts.evaluation.gaia_scorer import question_scorer

results["is_correct"] = results.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)

In [4]:
results["is_correct"].mean()

0.0033333333333333335

In [5]:
results.loc[results["is_correct"], ["question", "prediction"]]

,question,prediction
278,Reverse this ascii picture so that the fish is facing the opposite direction:\n>>$()>. Return the characters (without quotes) in a comma separated list.,".,>,),(,$,>,>"


In [6]:
def get_elements(x):
    output = x[0]["task"]
    for y in x[1:]:
        try:
            if "observation" in y:
                output += y["llm_output"] + "\nObservation:" + y["observation"]
            else:
                output += y["llm_output"] + "\Error:" + str(y["error"])
        except:
            pass
    return output


results["reasoning_trace"] = results["intermediate_steps"].apply(
    lambda x: get_elements(x)
)

In [7]:
results = results[["question", "prediction", "reasoning_trace"]].rename(
    columns={"prediction": "model_answer"}
)

In [8]:
results = results.merge(eval_df[["task_id", "question"]], on="question")
results = results.drop_duplicates(subset=["task_id"])
print(len(results))

300


In [9]:
results = results[
    ["task_id", "model_answer"]
]  # ["task_id", "model_answer", "reasoning_trace"]

In [11]:
results.to_json(
    f"gaia_{SET}_transformers_agents_gpt4o.jsonl", orient="records", lines=True
)

# Test blog

In [11]:
%reload_ext autoreload
%autoreload 2

In [1]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

/Users/aymeric/venvs/disposable/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
!pip install sentence-transformers faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.0 MB/s eta 0:00:00a 0:00:01


In [2]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

docs_processed = RecursiveCharacterTextSplitter(chunk_size=500).split_documents(
    source_docs
)[:1000]

embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(documents=docs_processed, embedding=embedding_model)

/Users/aymeric/venvs/disposable/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/aymeric/venvs/disposable/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
all_sources = list(set([doc.metadata["source"] for doc in docs_processed]))

In [5]:
from retriever_tool import RetrieverTool

retriever_tool = RetrieverTool(vectordb, all_sources)
print(retriever_tool(query="What is the capital of France?"))
retriever_tool.push_to_hub(repo_id="m-ric/langchain-retriever-tool")

Retrieved documents:

at+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct)
===Document===
BART](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+wa
===Document===
В области NLP ( Обработка текстов на естественном языке ):
- [Маскированное заполнение слов с помощью BERT](https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France)
- [Распознавание сущностей с

CommitInfo(commit_url='https://huggingface.co/spaces/m-ric/langchain-retriever-tool/commit/d1a0e494ff680e188cf41ab23194bfde5f4046ef', commit_message='Upload tool', commit_description='', oid='d1a0e494ff680e188cf41ab23194bfde5f4046ef', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
from transformers.agents import Tool
from langchain_core.vectorstores import VectorStore
import json


all_sources = list(set([doc.metadata["source"] for doc in docs_processed]))


class RetrieverTool(Tool):
    name = "retriever"
    description = "Retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "text",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        },
        "source": {
            "type": "text",
            "description": f"The source of the documents to search, as a str representation of a list. Possible values in the list are: {all_sources}. If this argument is not provided, all sources will be searched.",
        },
    }
    output_type = "text"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str, source: str = str(all_sources)) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        if isinstance(source, str) and "[" not in str(
            source
        ):  # if the source is not representing a list
            source = [source]
        source = json.loads(str(source).replace("'", '"'))

        docs = self.vectordb.similarity_search(query, filter={"source": source}, k=3)

        if len(docs) == 0:
            return "No documents found with this filtering. Try removing the source filter."
        return "Retrieved documents:\n\n" + "\n===Document===\n".join(
            [doc.page_content for doc in docs]
        )

In [56]:
RetrieverTool(vectordb)(query="What is the capital of France?")

'Retrieved documents:\n\nat+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct)\n===Document===\nBART](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+wa\n===Document===\nВ области NLP ( Обработка текстов на естественном языке ):\n- [Маскированное заполнение слов с помощью BERT](https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France)\n- [Распознавание су

In [12]:
from transformers.agents import Tool

retriever_tool = Tool.from_hub(
    repo_id="m-ric/langchain-retriever-tool", vectordb=vectordb, all_sources=all_sources
)

other_tool = Tool.from_hub(repo_id="m-ric/hf-model-downloads")

{'vectordb': <langchain_community.vectorstores.faiss.FAISS object at 0x3acdc95d0>, 'all_sources': ['datasets', 'evaluate', 'blog', 'hf-endpoints-documentation', 'diffusers', 'datasets-server', 'transformers', 'optimum', 'pytorch-image-models', 'gradio', 'deep-rl-class', 'course', 'peft', 'hub-docs']}
{}


In [10]:
retriever_tool(query="What is the capital of France?")

'Retrieved documents:\n\nat+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct)\n===Document===\nBART](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+wa\n===Document===\nВ области NLP ( Обработка текстов на естественном языке ):\n- [Маскированное заполнение слов с помощью BERT](https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France)\n- [Распознавание су

In [13]:
from transformers.agents import HfEngine, ReactJsonAgent

llm_engine = HfEngine("meta-llama/Meta-Llama-3-70B-Instruct")

agent = ReactJsonAgent(
    tools=[RetrieverTool(vectordb, all_sources)], llm_engine=llm_engine
)

agent_output = agent.run("Please show me a LORA finetuning script")

print("Final output:")
print(agent_output)

======== New task ========
Please show me a LORA finetuning script
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script', 'source': "['transformers', 'hf-endpoints-documentation']"}
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script'}
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script', 'source': 'github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py'}
Calling tool: 'final_answer' with arguments: {'answer': 'https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py'}


Final output:
https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py


In [1]:
!pip install lavague

  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached gradio_client-0.15.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 7.4 MB/s eta 0:00:00
  Using cached lxml-5.2.2-cp310-cp310-macosx_10_9_universal2.whl.metadata (3.4 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.4 MB/s eta 0:00:0031m27.6 MB/s eta 0:00:01
Using cached gradio_client-0.15.1-py3-none-any.whl (313 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 30.3 MB/s eta 0:00:00
Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 27.4 MB/s eta 0:00:00 

In [3]:
from lavague.drivers.selenium import SeleniumDriver
from lavague.core import ActionEngine, WorldModel
from lavague.core.agents import WebAgent

# Set up our three key components: Driver, Action Engine, World Model
driver = SeleniumDriver(headless=False)
action_engine = ActionEngine(driver)
world_model = WorldModel()

# Create Web Agent
agent = WebAgent(world_model, action_engine)

# Set URL
agent.get(
    "https://www.researchgate.net/publication/381296514_Spirituality_Culture_In_Organization_and_Impact_on_Employee_Well-Being"
)

# Run agent with a specific objective
agent.run("Give me the content of page 74 from the paper")

2024-07-02 17:42:25,315 - INFO - Screenshot folder cleared
2024-07-02 17:42:31,103 - INFO - Thoughts:
- The current screenshot shows a page from ResearchGate with a paper titled "Spirituality Culture In Organization and Impact on Employee Well-Being."
- The objective is to provide the content of page 74 from this paper.
- The paper is available for download as a PDF.
- The next step is to download the full-text PDF to access page 74.

Next engine: Navigation Engine
Instruction: Click on the "Download full-text PDF" button.
2024-07-02 17:42:46,978 - ERROR - Navigation error: Message: element click intercepted: Element <a href="https://www.researchgate.net/profile/Ashish-Waghe/publication/381296514_Spirituality_Culture_In_Organization_and_Impact_on_Employee_Well-Being/links/6666bf5785a4ee7261b387e8/Spirituality-Culture-In-Organization-and-Impact-on-Employee-Well-Being.pdf" class="nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--

ActionResult(instruction=None, code='from selenium import webdriver\nfrom selenium.webdriver.common.by import By\nfrom selenium.webdriver.chrome.options import Options\nfrom selenium.webdriver.common.keys import Keys\nfrom selenium.webdriver.common.action_chains import ActionChains\n\nelse:\nchrome_options = Options()\nchrome_options.add_argument("--no-sandbox")\nchrome_options.page_load_strategy = (\n)\n\ndriver = webdriver.Chrome(options=chrome_options)\n\ndriver.set_window_size(1080, 1080)\nviewport_height = driver.execute_script("return window.innerHeight;")\nheight_difference = 1080 - viewport_height\ndriver.set_window_size(1080, 1080 + height_difference)\n\ndriver.get("https://www.researchgate.net/publication/381296514_Spirituality_Culture_In_Organization_and_Impact_on_Employee_Well-Being")\n[\n    {\n        "action": {\n            "name": "click",\n            "args": {\n                "xpath": "/html/body/div[1]/div/div/div/div/div[2]/button[3]"\n            }\n        }\n  